## GiLOT

In [ ]:
model_name = "llama-7b"
beams = 200
max_new_tokens = 10

### LLM Attributions

In [ ]:
model, tokenizer, block_name, embedding_name, embed_token_name, _, _ = load_model(model_name)
interpreter = Interpreter(model, block_name, embedding_name, embed_token_name)
template = get_template(model_name)


query = "Give three tips for staying healthy."
input_text = f"{template['prefix']}{query.strip()}{template['postfix']}"
inputs = tokenizer(input_text, return_tensors="pt")
inputs.to(0)
attributions, probs_sequences = interpreter.interpret_ours(inputs.input_ids, args.beams, args.max_new_tokens,\
                                                            "optimal_transport")

### Visualizations

In [ ]:
from captum.attr import visualization

def show_heatmap_on_text(text, text_encoding, R_text):
    CLS_idx = text_encoding.argmax(dim=-1)
    R_text = R_text[CLS_idx, 1:CLS_idx]
    text_scores = R_text / R_text.sum()
    text_scores = text_scores.flatten()
    text_tokens=_tokenizer.encode(text)
    text_tokens_decoded=[tokenizer.decode([a]) for a in text_tokens]
    vis_data_records = [visualization.VisualizationDataRecord(text_scores,0,0,0,0,0,text_tokens_decoded,1)]
    visualization.visualize_text(vis_data_records)

In [ ]:
show_heatmap_on_text(input_text, inputs.input_ids, attributions)